## Setup

In [6]:
import sys
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
sys.path.append('../SignalBacktesting/') # god this is awful
from backtesting import backtesting
import cufflinks as cf
cf.set_config_file(offline=True, offline_show_link=False, theme='white', sharing='public', colorscale='ggplot')
from betterMT5 import connected
from datetime import datetime
import pandas as pd

In [2]:
with connected():
    test = backtesting.Backtest("../SignalBacktesting/chats/results_daniel.json", verbose=None)

In [3]:
with connected():
    results = test.run()

In [9]:
def gen_results(test_run = test.run_results, ignore = None):
    results = pd.DataFrame(test.make_results(test_run, ignore=ignore))
    results.drop(results[results.result>4].index, axis=0, inplace=True)
    results.drop(results[results.sl_pips>50].index, axis=0, inplace=True)
    results.drop(results[results.sl_pips<5].index, axis=0, inplace=True)
    results.open = pd.to_datetime(results.open.apply(str), utc=True)
    results.close = pd.to_datetime(results.close.apply(str), utc=True)
    return results

In [10]:
def gen_multi(results: list, names: list, attr: str = 'result'):
    to_pass = {k: v.set_index('open') for k, v in zip(names, results)}
    return pd.concat(to_pass, axis=0).reset_index()

def get_attr_multi(multi, attr):
    return multi.pivot('open', 'level_0', attr)

## Results (Vanilla Daniel Savage)

In [11]:
results = gen_results()
data_loss = len(test.trades) - results.shape[0]
print(f'Post-adjustment data loss: {data_loss} ({data_loss/len(test.trades):.2%})')

Post-adjustment data loss: 21 (4.90%)


In [12]:
results.iplot(kind='scatter', mode='markers', x='close', title='Results (R) by order type', yTitle='R',
              y='result', categories='type', size=6, colors='red blue purple green grey black'.split(' '))

In [14]:
results_ignore_close = gen_results(ignore=['CLOSE'])
results_ignore_sl_to_be = gen_results(ignore=['SL_TO_BE'])
results_ignore_partials = gen_results(ignore=['PARTIALS'])
multi = gen_multi(results=[results, results_ignore_close, results_ignore_sl_to_be, results_ignore_partials],
                  names='normal ignore_close ignore_be ignore_partials'.split(' '))


In [15]:
interpolated = get_attr_multi(multi, 'result').cumsum().resample('3600T').mean().interpolate(method='spline', order=3)
interpolated.iplot(kind='scatter', title='Cumulative results by variant (R)', yTitle='R',
    xTitle='date', opacity=1, zerolinecolor='black')
del interpolated

In [16]:
get_attr_multi(multi, 'result').groupby(pd.Grouper(freq='M')).sum().iplot(kind='bar', barmode='group',
    title='Results by variant by month (R)', xTitle='date', yTitle='R', opacity=1)

In [17]:
get_attr_multi(multi, 'result').groupby(pd.Grouper(freq='D')).sum().iplot(kind='bar', subplots=True,
    title='Results by day (R) by variant', xTitle='date', yTitle='R', opacity=1, bargroupgap=0.1)

In [18]:
df = get_attr_multi(multi, 'result').groupby(pd.Grouper(freq='D')).count()
df.loc[(df!=0).any(axis=1)].iplot(kind='hist', title='Number of trades in 1 day (count)', 
    xTitle='number of trades', yTitle='frequency', opacity=1, bargroupgap=0.6, linecolor='#F2F3F5')

In [19]:
df = get_attr_multi(multi, 'result').groupby(
    pd.Grouper(freq='D')).agg(['count', 'sum']).reset_index(drop=True).stack().to_dict()
final_d = {}
for n in range(1, 7):
    final_d[n] = dict()
    for k, v in df.items():
        # ex: ignore_be:
        final_d[n][k] = 0
        flag = False
        for k2, v2 in v.items():
            if k2[1] == 'count':
                if v2 == n:
                    flag = True
                    continue
                flag = False
            if flag:
                final_d[n][k] += v2

pd.DataFrame(final_d).T.iplot(kind='bar', title='Cumulative result by number of trades in 1 day (R)',
    xTitle='number of trades', yTitle='R', opacity=1, linecolor='#F2F3F5', bargap=0.3)

In [70]:
get_attr_multi(multi, 'sl_pips').iplot(kind='hist', barmode='overlay', title='SL (pips) distribution overlay', 
    yTitle='frequency', xTitle='pips', opacity=1, linecolor='white')

In [21]:
df = get_attr_multi(multi, 'close')
df = df.apply(lambda x: (x-df.index).dt.total_seconds()/60).reset_index().drop('open', axis=1)
df.iplot(kind='hist', barmode='stack', title='Trade length (minutes) distribution stack', yTitle='amount', 
    xTitle='minutes', opacity=1, linecolor='#F2F3F5')

### Let's try to limit the max number of trades per day

In [91]:
new_results = results.set_index('open')
new_results['counter'] = 1
new_results['counter'] = new_results[['counter']].groupby(pd.Grouper(freq='D')).transform(lambda x: x.cumsum())
new_results = new_results[new_results.counter < 3]
new_results = new_results.resample('T').sum()
new_results['new'] = new_results.resample('T').sum()['result']
new_results['old'] = results.set_index('open').resample('T').sum()['result']
new_results = new_results[['new', 'old']].loc[(new_results.new != 0) | (new_results.old != 0)]
new_results.new.loc[new_results.new==0] = None

In [90]:
df = new_results.groupby(pd.Grouper(freq='D')).count()
df.loc[(df!=0).any(axis=1)].iplot(kind='hist', title='Number of trades in 1 day (count)', barmode='group',
    xTitle='number of trades', yTitle='frequency', opacity=1, bargroupgap=0, linecolor='#F2F3F5')


In [97]:
interpolated = new_results.cumsum().resample('3600T').mean().interpolate(method='spline', order=3)
interpolated.iplot(kind='scatter', title='Cumulative results (R)', yTitle='R',
    xTitle='date', opacity=1, zerolinecolor='black')
del interpolated

In [96]:
new_results.groupby(pd.Grouper(freq='M')).sum().iplot(kind='bar', barmode='group',
    title='Results by month (R)', xTitle='date', yTitle='R', opacity=1)

In [98]:
new_results.groupby(pd.Grouper(freq='D')).sum().iplot(kind='bar', subplots=True,
    title='Results by day (R)', xTitle='date', yTitle='R', opacity=1, bargroupgap=0.1)